In [1]:
import getpass
import os

from dotenv import load_dotenv

# Load the .env file
_ = load_dotenv()

* Duckdb

DuckDB is a relational (table-oriented) DBMS that supports the Structured Query Language (SQL).

In [2]:
import duckdb

In [3]:
db_path="../data/inventory.db"
with duckdb.connect(db_path) as con:
       res = con.sql("SELECT * FROM Stock where id = '1009AA';")
       print(res)

┌─────────┬────────────────────────┬─────────┬────────────────────┬────────────────────────┬────────────┐
│   id    │ current_stock_quantity │  units  │ avg_lead_time_days │ maximum_lead_time_days │ unit_price │
│ varchar │         double         │ varchar │       int64        │         int64          │   double   │
├─────────┼────────────────────────┼─────────┼────────────────────┼────────────────────────┼────────────┤
│ 1009AA  │                 7210.0 │ Kg      │                 30 │                     48 │   28.76326 │
└─────────┴────────────────────────┴─────────┴────────────────────┴────────────────────────┴────────────┘



* Model

In [4]:
from agno.models.huggingface import HuggingFace
from agno.models.groq import Groq
from agno.models.ollama import Ollama

hg_model=HuggingFace(
        id="meta-llama/Meta-Llama-3-8B-Instruct",
        api_key=os.getenv("HUGGINGFACE_API_TOKEN"),
    )

groq_model=Groq(id="llama-3.1-8b-instant",api_key=os.getenv("GROQ_API_KEY")),

# Define the Agent

ollama_model=Ollama(id="llama3.2:3b")

### DuckDb

DuckDbTools enable an Agent to run SQL and analyze data using DuckDb.

In [5]:
# from agno.agent import Agent
# from agno.tools.duckdb import DuckDbTools

# sql_agent = Agent(
#     model=ollama_model,
#     tools=[DuckDbTools( db_path=db_path)],
#     stream=False,
#     markdown=False
# )


In [6]:
# sql_agent.print_response("What is the price of the product with id 1009AA in the stock table?")


### Sqlalchemy

In [7]:

from sqlalchemy.engine import create_engine

db_url = 'sqlite:///../data/inventory.db'
# Create a database engine
db_engine = create_engine(db_url)

print("dialect:",db_engine.dialect)


dialect: <sqlalchemy.dialects.sqlite.pysqlite.SQLiteDialect_pysqlite object at 0x000001CF868E5FD0>


* Realizar consultas

In [8]:
from sqlalchemy import text
# Obtener un objeto Connection
conn = db_engine.connect()

# Crear un objeto Text con la consulta SQL
query = text("SELECT * FROM Stock LIMIT 1;")

# Ejecutar la consulta
result = conn.execute(query)

# Imprimir los resultados
for row in result:
    print(row)

# No olvides cerrar la conexión cuando termines
conn.close()

('1009AA', 7210.0, 'Kg', 30, 48, 28.76326)


In [9]:
from sqlalchemy import text
# Obtener un objeto Connection
conn = db_engine.connect()

# Crear un objeto Text con la consulta SQL
query = text("SELECT * FROM Orders LIMIT 1;")

# Ejecutar la consulta
result = conn.execute(query)

# Imprimir los resultados
for row in result:
    print(row)

# No olvides cerrar la conexión cuando termines
conn.close()

('2022-11-19 00:00:00.000000', '3084CA', 1.0)


In [10]:
from sqlalchemy import text
# Obtener un objeto Connection
conn = db_engine.connect()

# Crear un objeto Text con la consulta SQL
query = text("SELECT * FROM Stock LIMIT 1;")

# Ejecutar la consulta
result = conn.execute(query)

# Imprimir los resultados
for row in result:
    print(row)

# No olvides cerrar la conexión cuando termines
conn.close()

('1009AA', 7210.0, 'Kg', 30, 48, 28.76326)


* Crear agente

In [11]:
from agno.tools.sql import SQLTools
from agno.agent import Agent

agent = Agent(model=hg_model, tools=[SQLTools(db_engine=db_engine)])

In [27]:
agent.print_response("Tell me about the stock table",markdown=True)

Output()

HfHubHTTPError: 422 Client Error: Unprocessable Entity for url: https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3-8B-Instruct/v1/chat/completions (Request ID: OYA2Cr)

Input validation error: cannot compile regex from schema: 'type' must be a string
Make sure 'text-generation' task is supported by the model.